In [26]:
from flask import Flask, render_template,request,jsonify
import csv
import pandas as pd
import ast
import numpy as np
import tensorflow as tf
app = Flask(__name__)


@app.route('/')
def hello():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        # check if the post request has the file part
        if 'file' not in request.files:
            return 'No file part'
        file = request.files['file']
        # if user does not select file, browser also
        # submit an empty part without filename
        if file.filename == '':
            return 'No selected file'
        if file and file.filename.endswith('.csv'):
            # Here, you can do something with the uploaded CSV file
            df = pd.read_csv(file,sep='\t')
            dfdata=df['interval']
            
            interval_arrays = df['interval'].apply(lambda x: np.array(ast.literal_eval(x)))
            dfdata_filtered = []
            
            for row in dfdata:
                arr_str = row  # Assuming row is a string representation of the array
                arr = np.fromstring(arr_str[1:-1], sep=',')  # Convert string to numpy array
                if arr.size > 0:  # Skip empty or singleton arrays
                    dfdata_filtered.append(arr.reshape( 100))
            # For example, you can read the CSV file and perform prediction
            # For now, let's just return a message indicating success
            # Create input data and labels
            inputs = dfdata_filtered[:-1]  # All elements except the last one
            labels = dfdata_filtered[1:]   # All elements except the first one

            # Define a function to generate batches
            def generate_batches(inputs, labels, batch_size):
                # Combine inputs and labels into a dataset
                dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))

                # Batch the dataset
                dataset = dataset.batch(batch_size)

                return dataset

            # Example usage:
            batch_size = 1  # Set your desired batch size

            # Generate the dataset
            dataset = generate_batches(inputs, labels, batch_size)
            
            model= tf.keras.models.load_model('model.h5')
            
            pred=model.predict(dataset)
            
            pred = pred.tolist()
            energy_list=df['E0'].tolist()
            energy_list = energy_list[:-1]
            dfrnn = pd.DataFrame({'E0': energy_list, 'interval': pred})
            dfrnn.to_csv('pred.csv', index=False)
            with open('pred.csv', newline='') as csvfile:
                csvreader = csv.reader(csvfile)
                data = list(csvreader)

            # Render data in HTML template
            return render_template('pred.html', data=data)

        else:
            return 'Please upload a CSV file'
        
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Apr/2024 11:25:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2024 11:25:28] "GET /static/style.css HTTP/1.1" 404 -
127.0.0.1 - - [03/Apr/2024 11:25:29] "POST /predict HTTP/1.1" 200 -


41651/41651 [==============================] - 164s 4ms/step


127.0.0.1 - - [03/Apr/2024 11:28:36] "POST /predict HTTP/1.1" 200 -


In [2]:
%tb

No traceback available to show.
